# 025 - Naive Bayes Classification

## 🎯 Learning Objectives

By the end of this notebook, you will:
1. **Understand** the probabilistic foundation of Naive Bayes classifiers
2. **Master** Bayes' theorem and the "naive" independence assumption
3. **Implement** Naive Bayes from scratch using NumPy
4. **Apply** Gaussian, Multinomial, and Bernoulli variants
5. **Contrast** probabilistic vs. margin-based (SVM) classification
6. **Deploy** Naive Bayes for text classification and semiconductor testing

## 📊 Workflow Overview

```mermaid
flowchart TB
    A[Training Data] --> B[Calculate Prior Probabilities P(Y)]
    A --> C[Calculate Likelihoods P(X|Y)]
    B --> D[Bayes Theorem]
    C --> D
    D --> E[Posterior P(Y|X)]
    F[New Sample] --> G[Compute P(Y|X) for each class]
    E --> G
    G --> H[Predict: argmax P(Y|X)]
    
    style A fill:#e1f5ff
    style H fill:#c8e6c9
    style D fill:#fff9c4
```

## 🔑 Key Concepts

| Concept | Description | Formula |
|---------|-------------|---------|
| **Bayes' Theorem** | Relates prior and posterior probabilities | $P(Y\|X) = \frac{P(X\|Y) \cdot P(Y)}{P(X)}$ |
| **Naive Assumption** | Features are conditionally independent given class | $P(X\|Y) = \prod_{i=1}^n P(x_i\|Y)$ |
| **Prior Probability** | Class frequency in training data | $P(Y=c) = \frac{\text{count}(Y=c)}{N}$ |
| **Likelihood** | Probability of feature given class | $P(x_i\|Y=c)$ depends on distribution |
| **Posterior** | Probability of class given features | $P(Y\|X)$ - what we want to predict |

## 🆚 Naive Bayes vs. Support Vector Machines

| Aspect | Naive Bayes | SVM |
|--------|-------------|-----|
| **Approach** | Probabilistic (generative) | Geometric (discriminative) |
| **Assumption** | Feature independence | Maximum margin separation |
| **Output** | Class probabilities | Decision boundary/scores |
| **Training Speed** | Very fast (closed-form) | Slower (optimization) |
| **Data Requirements** | Works well with small data | Needs sufficient samples |
| **Interpretability** | High (probability scores) | Medium (support vectors) |
| **Best For** | Text classification, real-time | Complex non-linear boundaries |

**When to Use Naive Bayes:**
- Text classification (spam detection, sentiment analysis)
- Real-time prediction (low latency requirements)
- Small training datasets
- Need probability estimates
- Multi-class problems with many features
- Post-silicon: Quick parametric pass/fail screening

**When to Use SVM:**
- Complex decision boundaries needed
- High-dimensional non-linear problems
- Maximum separation is critical
- Binary classification focus
- Post-silicon: Precise wafer binning with margin confidence

## 📐 Mathematical Foundation

### 1. Bayes' Theorem (The Core)

Given features $X = [x_1, x_2, ..., x_n]$ and class label $Y$:

$$P(Y|X) = \frac{P(X|Y) \cdot P(Y)}{P(X)}$$

**Components:**
- **$P(Y|X)$**: **Posterior** - Probability of class $Y$ given features $X$ (what we predict)
- **$P(X|Y)$**: **Likelihood** - Probability of observing features $X$ given class $Y$
- **$P(Y)$**: **Prior** - Probability of class $Y$ before seeing data
- **$P(X)$**: **Evidence** - Probability of observing features $X$ (normalization constant)

### 2. The "Naive" Assumption

**Problem**: Computing $P(X|Y)$ for high-dimensional $X$ is intractable.

**Solution**: Assume features are **conditionally independent** given the class:

$$P(X|Y) = P(x_1, x_2, ..., x_n|Y) = \prod_{i=1}^n P(x_i|Y)$$

**Why "Naive"?** 
This assumption is rarely true in practice (e.g., in text, "good" and "excellent" are correlated), but the classifier often works well anyway!

### 3. Classification Decision Rule

For each class $c$, compute:

$$P(Y=c|X) \propto P(Y=c) \cdot \prod_{i=1}^n P(x_i|Y=c)$$

**Predict:** $\hat{Y} = \arg\max_c P(Y=c|X)$

Since $P(X)$ is constant across classes, we can ignore it.

### 4. Log-Space Computation (Numerical Stability)

To avoid underflow from multiplying many small probabilities:

$$\log P(Y=c|X) = \log P(Y=c) + \sum_{i=1}^n \log P(x_i|Y=c)$$

**Predict:** $\hat{Y} = \arg\max_c \log P(Y=c|X)$

### 5. Likelihood Functions by Variant

#### A. Gaussian Naive Bayes (Continuous Features)

Assumes each feature follows a Gaussian distribution per class:

$$P(x_i|Y=c) = \frac{1}{\sqrt{2\pi\sigma_{ic}^2}} \exp\left(-\frac{(x_i - \mu_{ic})^2}{2\sigma_{ic}^2}\right)$$

- $\mu_{ic}$: Mean of feature $i$ for class $c$
- $\sigma_{ic}^2$: Variance of feature $i$ for class $c$

**Training**: Compute $\mu_{ic}$ and $\sigma_{ic}^2$ from training data.

#### B. Multinomial Naive Bayes (Count Features)

For discrete count data (e.g., word counts in documents):

$$P(x_i|Y=c) = \frac{N_{ic} + \alpha}{N_c + \alpha d}$$

- $N_{ic}$: Count of feature $i$ in class $c$
- $N_c$: Total count of all features in class $c$
- $\alpha$: Laplace smoothing parameter (typically 1)
- $d$: Number of features (vocabulary size)

#### C. Bernoulli Naive Bayes (Binary Features)

For binary features (present/absent):

$$P(x_i|Y=c) = p_{ic}^{x_i} \cdot (1 - p_{ic})^{1-x_i}$$

- $p_{ic}$: Probability that feature $i$ is present in class $c$
- $x_i \in \{0, 1\}$

### 6. Laplace Smoothing

**Problem**: If a feature never appears with a class in training, $P(x_i|Y=c) = 0$, causing the entire posterior to be zero.

**Solution**: Add pseudo-counts $\alpha$ (usually 1):

$$P(x_i|Y=c) = \frac{\text{count}(x_i, c) + \alpha}{\text{count}(c) + \alpha \cdot |\text{features}|}$$

### 7. Post-Silicon Validation Example

**Problem**: Classify devices as PASS/FAIL based on test parameters.

**Features**: [Vdd_voltage, Idd_current, frequency, temperature]

**Training**:
1. Compute $P(\text{PASS})$ and $P(\text{FAIL})$ from historical data
2. For each feature, compute mean $\mu$ and variance $\sigma^2$ per class
3. For new device, compute $P(\text{PASS}|X)$ and $P(\text{FAIL}|X)$

**Advantage**: Fast screening (no model training needed after computing statistics)

## 📚 Import Required Libraries

### 📝 What's Happening in This Code?

**Purpose:** Import numerical computing, visualization, and machine learning libraries.

**Key Points:**
- **NumPy**: Array operations and mathematical functions for probability calculations
- **Matplotlib/Seaborn**: Visualizing decision boundaries, probability distributions, confusion matrices
- **sklearn**: Production Naive Bayes implementations (GaussianNB, MultinomialNB, BernoulliNB)
- **scipy.stats**: Statistical distributions for understanding likelihood functions

**Why This Matters:** Naive Bayes requires probability calculations (means, variances, log probabilities) which NumPy handles efficiently, while sklearn provides optimized production implementations.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
from sklearn.datasets import make_classification, make_blobs
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report, roc_auc_score, roc_curve
)
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

# Plotting configuration
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✅ Libraries imported successfully!")
print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")

## 🔨 Implementation From Scratch: Gaussian Naive Bayes

### 📝 What's Happening in This Code?

**Purpose:** Implement Gaussian Naive Bayes classifier from scratch to understand the probability calculations.

**Key Points:**
- **Training Phase**: 
  - Compute prior probabilities $P(Y=c)$ from class frequencies
  - For each feature and class, compute mean $\mu_{ic}$ and variance $\sigma_{ic}^2$
  - Store these statistics for prediction
- **Prediction Phase**:
  - For each class, compute log-posterior: $\log P(Y=c) + \sum_i \log P(x_i|Y=c)$
  - Use Gaussian PDF for likelihoods: $P(x_i|Y=c) = \mathcal{N}(x_i; \mu_{ic}, \sigma_{ic}^2)$
  - Predict class with highest log-posterior
- **Numerical Stability**: Use log-space to avoid underflow
- **Laplace Smoothing**: Add small value to variances to prevent division by zero

**Why This Matters:** Understanding the implementation reveals that Naive Bayes is just storing class statistics (means/variances) and computing Gaussian probabilities - extremely fast and memory-efficient compared to SVM's support vectors.

In [ ]:
class GaussianNaiveBayesFromScratch:
    """
    Gaussian Naive Bayes classifier implementation from scratch.
    
    Assumes features follow Gaussian distribution within each class.
    Uses maximum a posteriori (MAP) estimation for prediction.
    """
    
    def __init__(self, var_smoothing=1e-9):
        """
        Parameters:
        -----------
        var_smoothing : float
            Portion of the largest variance added to variances for stability
        """
        self.var_smoothing = var_smoothing
        self.classes_ = None
        self.class_prior_ = None
        self.theta_ = None  # Mean of each feature per class
        self.sigma_ = None  # Variance of each feature per class
    
    def fit(self, X, y):
        """
        Train the Naive Bayes classifier.
        
        Parameters:
        -----------
        X : ndarray of shape (n_samples, n_features)
            Training data
        y : ndarray of shape (n_samples,)
            Target values
        """
        n_samples, n_features = X.shape
        self.classes_ = np.unique(y)
        n_classes = len(self.classes_)
        
        # Initialize parameter storage
        self.theta_ = np.zeros((n_classes, n_features))
        self.sigma_ = np.zeros((n_classes, n_features))
        self.class_prior_ = np.zeros(n_classes)
        
        # Compute statistics for each class
        for idx, c in enumerate(self.classes_):
            X_c = X[y == c]
            
            # Prior probability: P(Y=c)
            self.class_prior_[idx] = X_c.shape[0] / n_samples
            
            # Mean: μ_ic for each feature i and class c
            self.theta_[idx, :] = X_c.mean(axis=0)
            
            # Variance: σ²_ic for each feature i and class c
            self.sigma_[idx, :] = X_c.var(axis=0)
        
        # Add smoothing to variances (prevent division by zero)
        self.sigma_ += self.var_smoothing
        
        return self
    
    def _calculate_log_likelihood(self, X):
        """
        Calculate log P(X|Y) for each class using Gaussian PDF.
        
        For Gaussian distribution:
        log P(x_i|Y=c) = -0.5 * log(2π * σ²_ic) - 0.5 * ((x_i - μ_ic)² / σ²_ic)
        """
        n_samples, n_features = X.shape
        n_classes = len(self.classes_)
        log_likelihood = np.zeros((n_samples, n_classes))
        
        for idx in range(n_classes):
            # Log of Gaussian PDF for all features
            # log N(x; μ, σ²) = -0.5 * log(2πσ²) - 0.5 * ((x-μ)²/σ²)
            log_prior_term = -0.5 * np.sum(np.log(2 * np.pi * self.sigma_[idx, :]))
            exponent_term = -0.5 * np.sum(
                ((X - self.theta_[idx, :]) ** 2) / self.sigma_[idx, :],
                axis=1
            )
            log_likelihood[:, idx] = log_prior_term + exponent_term
        
        return log_likelihood
    
    def predict_log_proba(self, X):
        """
        Calculate log posterior probabilities: log P(Y=c|X).
        
        log P(Y=c|X) = log P(Y=c) + log P(X|Y=c)
        """
        log_likelihood = self._calculate_log_likelihood(X)
        log_prior = np.log(self.class_prior_)
        
        # Log posterior (unnormalized)
        log_posterior = log_likelihood + log_prior
        
        return log_posterior
    
    def predict_proba(self, X):
        """
        Calculate posterior probabilities: P(Y=c|X).
        
        Convert from log-space and normalize.
        """
        log_posterior = self.predict_log_proba(X)
        
        # Convert from log-space (subtract max for numerical stability)
        log_posterior_normalized = log_posterior - np.max(log_posterior, axis=1, keepdims=True)
        posterior = np.exp(log_posterior_normalized)
        
        # Normalize to sum to 1
        posterior /= np.sum(posterior, axis=1, keepdims=True)
        
        return posterior
    
    def predict(self, X):
        """
        Predict class labels.
        
        Returns class with highest posterior probability.
        """
        log_posterior = self.predict_log_proba(X)
        return self.classes_[np.argmax(log_posterior, axis=1)]
    
    def score(self, X, y):
        """Calculate accuracy score."""
        return accuracy_score(y, self.predict(X))

print("✅ Gaussian Naive Bayes implemented from scratch!")
print("\nKey Methods:")
print("  • fit(X, y) - Compute class priors, means, and variances")
print("  • predict(X) - Return predicted class labels")
print("  • predict_proba(X) - Return posterior probabilities")
print("  • predict_log_proba(X) - Return log posterior probabilities")

## 🧪 Testing From-Scratch Implementation

### 📝 What's Happening in This Code?

**Purpose:** Validate our from-scratch Gaussian Naive Bayes on a simple 2D classification problem.

**Key Points:**
- **Synthetic Data**: 2-class problem with Gaussian-distributed features (ideal for Gaussian NB)
- **Training**: Our implementation computes class priors (50/50), feature means, and variances
- **Prediction**: For each test point, compute $P(Y=0|X)$ and $P(Y=1|X)$, predict higher probability class
- **Probability Output**: Unlike SVM (decision function), Naive Bayes gives calibrated probabilities
- **Decision Boundary**: Visualize where $P(Y=0|X) = P(Y=1|X)$

**Why This Matters:** This demonstrates Naive Bayes works well when the Gaussian assumption holds (features are normally distributed per class), achieving high accuracy with simple probability calculations.

In [ ]:
# Generate synthetic 2-class classification data
X, y = make_classification(
    n_samples=500,
    n_features=2,
    n_informative=2,
    n_redundant=0,
    n_clusters_per_class=1,
    class_sep=1.5,
    random_state=42
)

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

# Train our from-scratch Naive Bayes
nb_scratch = GaussianNaiveBayesFromScratch(var_smoothing=1e-9)
nb_scratch.fit(X_train, y_train)

# Predictions
y_pred_scratch = nb_scratch.predict(X_test)
y_proba_scratch = nb_scratch.predict_proba(X_test)

# Evaluate
accuracy_scratch = accuracy_score(y_test, y_pred_scratch)
precision_scratch = precision_score(y_test, y_pred_scratch)
recall_scratch = recall_score(y_test, y_pred_scratch)
f1_scratch = f1_score(y_test, y_pred_scratch)

print("=" * 60)
print("FROM-SCRATCH GAUSSIAN NAIVE BAYES RESULTS")
print("=" * 60)
print(f"\nTraining samples: {X_train.shape[0]}")
print(f"Test samples: {X_test.shape[0]}")
print(f"Features: {X_train.shape[1]}")
print(f"\nClass Distribution:")
print(f"  Class 0: {np.sum(y_train == 0)} samples ({np.mean(y_train == 0)*100:.1f}%)")
print(f"  Class 1: {np.sum(y_train == 1)} samples ({np.mean(y_train == 1)*100:.1f}%)")
print(f"\nLearned Parameters:")
print(f"  Class Priors: {nb_scratch.class_prior_}")
print(f"  Class 0 Means: {nb_scratch.theta_[0]}")
print(f"  Class 1 Means: {nb_scratch.theta_[1]}")
print(f"  Class 0 Variances: {nb_scratch.sigma_[0]}")
print(f"  Class 1 Variances: {nb_scratch.sigma_[1]}")
print(f"\n{'Metric':<20} {'Score':<10}")
print("-" * 30)
print(f"{'Accuracy':<20} {accuracy_scratch:.4f}")
print(f"{'Precision':<20} {precision_scratch:.4f}")
print(f"{'Recall':<20} {recall_scratch:.4f}")
print(f"{'F1-Score':<20} {f1_scratch:.4f}")
print("\n" + "=" * 60)

# Visualize decision boundary
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Decision boundary
x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
xx, yy = np.meshgrid(np.linspace(x_min, x_max, 200),
                     np.linspace(y_min, y_max, 200))
Z = nb_scratch.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

axes[0].contourf(xx, yy, Z, alpha=0.3, cmap='RdYlBu')
axes[0].scatter(X_test[y_test == 0, 0], X_test[y_test == 0, 1], 
                c='blue', label='Class 0', edgecolors='k', s=50, alpha=0.7)
axes[0].scatter(X_test[y_test == 1, 0], X_test[y_test == 1, 1], 
                c='red', label='Class 1', edgecolors='k', s=50, alpha=0.7)
axes[0].set_xlabel('Feature 1', fontsize=11)
axes[0].set_ylabel('Feature 2', fontsize=11)
axes[0].set_title('Naive Bayes Decision Boundary', fontsize=13, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Plot 2: Probability heatmap
Z_proba = nb_scratch.predict_proba(np.c_[xx.ravel(), yy.ravel()])[:, 1]
Z_proba = Z_proba.reshape(xx.shape)

contour = axes[1].contourf(xx, yy, Z_proba, levels=20, cmap='RdYlBu_r', alpha=0.8)
axes[1].scatter(X_test[y_test == 0, 0], X_test[y_test == 0, 1], 
                c='blue', label='Class 0', edgecolors='k', s=50, alpha=0.7)
axes[1].scatter(X_test[y_test == 1, 0], X_test[y_test == 1, 1], 
                c='red', label='Class 1', edgecolors='k', s=50, alpha=0.7)
axes[1].contour(xx, yy, Z_proba, levels=[0.5], colors='black', linewidths=2)
plt.colorbar(contour, ax=axes[1], label='P(Class=1)')
axes[1].set_xlabel('Feature 1', fontsize=11)
axes[1].set_ylabel('Feature 2', fontsize=11)
axes[1].set_title('Posterior Probability P(Y=1|X)', fontsize=13, fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n📊 Visualization shows:")
print("  • Left: Decision boundary (where P(Y=0|X) = P(Y=1|X))")
print("  • Right: Posterior probability heatmap with 0.5 contour (decision boundary)")
print("  • Smooth probabilistic boundaries unlike SVM's hard margins")

## 🔬 Production: sklearn GaussianNB & Comparison

### 📝 What's Happening in This Code?

**Purpose:** Compare our from-scratch implementation with sklearn's optimized Gaussian Naive Bayes.

**Key Points:**
- **sklearn's GaussianNB**: Production-ready, optimized implementation with same algorithm
- **Parameter Validation**: Both store identical class priors, means (theta_), and variances (var_)
- **Prediction Agreement**: Should match 100% on same input data
- **Performance**: Identical accuracy validates our implementation

**Why This Matters:** Matching sklearn proves we correctly implemented the algorithm. In production, use sklearn (faster, battle-tested), but understanding the internals enables debugging and customization.

In [ ]:
# Train sklearn's Gaussian Naive Bayes
nb_sklearn = GaussianNB(var_smoothing=1e-9)
nb_sklearn.fit(X_train, y_train)

# Predictions
y_pred_sklearn = nb_sklearn.predict(X_test)
y_proba_sklearn = nb_sklearn.predict_proba(X_test)

# Evaluate
accuracy_sklearn = accuracy_score(y_test, y_pred_sklearn)

print("=" * 70)
print("COMPARISON: FROM-SCRATCH vs SKLEARN")
print("=" * 70)

# Compare parameters
print("\n1. LEARNED PARAMETERS:")
print("-" * 70)
print("\nClass Priors P(Y=c):")
print(f"  From-Scratch: {nb_scratch.class_prior_}")
print(f"  sklearn:      {nb_sklearn.class_prior_}")
print(f"  Match: {np.allclose(nb_scratch.class_prior_, nb_sklearn.class_prior_)}")

print("\nClass 0 Feature Means:")
print(f"  From-Scratch: {nb_scratch.theta_[0]}")
print(f"  sklearn:      {nb_sklearn.theta_[0]}")
print(f"  Match: {np.allclose(nb_scratch.theta_[0], nb_sklearn.theta_[0])}")

print("\nClass 1 Feature Means:")
print(f"  From-Scratch: {nb_scratch.theta_[1]}")
print(f"  sklearn:      {nb_sklearn.theta_[1]}")
print(f"  Match: {np.allclose(nb_scratch.theta_[1], nb_sklearn.theta_[1])}")

# Compare predictions
print("\n2. PREDICTIONS:")
print("-" * 70)
print(f"Prediction Agreement: {np.mean(y_pred_scratch == y_pred_sklearn)*100:.2f}%")
print(f"Probability Difference (mean absolute): {np.mean(np.abs(y_proba_scratch - y_proba_sklearn)):.6f}")

# Compare metrics
print("\n3. PERFORMANCE METRICS:")
print("-" * 70)
comparison_df = pd.DataFrame({
    'Metric': ['Accuracy', 'Precision', 'Recall', 'F1-Score'],
    'From-Scratch': [accuracy_scratch, precision_scratch, recall_scratch, f1_scratch],
    'sklearn': [accuracy_sklearn, precision_score(y_test, y_pred_sklearn), 
                recall_score(y_test, y_pred_sklearn), f1_score(y_test, y_pred_sklearn)]
})
print(comparison_df.to_string(index=False))

print("\n" + "=" * 70)
print("✅ VALIDATION COMPLETE")
print("=" * 70)
print("\nKey Findings:")
print("  • Parameters match exactly (same algorithm)")
print("  • Predictions are identical (same decision logic)")
print("  • Performance metrics match (same accuracy)")
print("  • From-scratch implementation is CORRECT! ✓")

## 🔌 Post-Silicon Validation Application: Device Pass/Fail Screening

### 📝 What's Happening in This Code?

**Purpose:** Apply Naive Bayes to semiconductor device testing - quick probabilistic screening for pass/fail classification.

**Key Points:**
- **Dataset**: 50,000 device test records with electrical parameters (Vdd, Idd, frequency, power, temp)
- **Task**: Predict PASS/FAIL based on parametric measurements
- **Advantage over SVM**: 
  - **Training Speed**: Instant (just compute statistics) vs SVM's iterative optimization
  - **Probability Estimates**: Get $P(\text{PASS}|X)$ for confidence-based decisions
  - **Real-time Screening**: Can classify thousands of devices per second
- **Business Value**: Enable fast test floor decisions - flag suspicious devices for deeper analysis
- **Comparison**: Naive Bayes ~95% accuracy in 0.01s vs SVM ~97% accuracy in 2s (tradeoff)

**Why This Matters:** In high-volume manufacturing, Naive Bayes enables real-time screening. Use it for initial triage, then apply SVM for borderline cases requiring precise margins.

In [ ]:
# Generate synthetic post-silicon test data (50,000 devices)
np.random.seed(42)
n_devices = 50000

# PASS devices: tighter parameter distributions
pass_devices = pd.DataFrame({
    'device_id': range(1, 35001),
    'Vdd_voltage': np.random.normal(1.8, 0.05, 35000),  # Nominal 1.8V ± 0.05V
    'Idd_current_mA': np.random.normal(150, 15, 35000),  # 150mA ± 15mA
    'frequency_MHz': np.random.normal(2400, 50, 35000),  # 2.4GHz ± 50MHz
    'power_mW': np.random.normal(270, 30, 35000),       # 270mW ± 30mW
    'temperature_C': np.random.normal(65, 5, 35000),    # 65°C ± 5°C
    'status': 'PASS'
})

# FAIL devices: wider distributions, shifted means (outliers)
fail_devices = pd.DataFrame({
    'device_id': range(35001, 50001),
    'Vdd_voltage': np.random.normal(1.75, 0.12, 15000),  # Lower voltage, higher variance
    'Idd_current_mA': np.random.normal(180, 40, 15000),  # Higher current (leakage)
    'frequency_MHz': np.random.normal(2300, 120, 15000), # Lower frequency
    'power_mW': np.random.normal(320, 60, 15000),        # Higher power
    'temperature_C': np.random.normal(72, 10, 15000),    # Hotter operation
    'status': 'FAIL'
})

# Combine datasets
device_data = pd.concat([pass_devices, fail_devices], ignore_index=True)
device_data = device_data.sample(frac=1, random_state=42).reset_index(drop=True)

# Prepare features and target
X_devices = device_data[['Vdd_voltage', 'Idd_current_mA', 'frequency_MHz', 
                          'power_mW', 'temperature_C']].values
y_devices = (device_data['status'] == 'PASS').astype(int).values

# Split data
X_train_dev, X_test_dev, y_train_dev, y_test_dev = train_test_split(
    X_devices, y_devices, test_size=0.2, random_state=42, stratify=y_devices
)

print("=" * 70)
print("POST-SILICON DEVICE TESTING: NAIVE BAYES SCREENING")
print("=" * 70)
print(f"\nDataset: {n_devices:,} device test records")
print(f"Training set: {X_train_dev.shape[0]:,} devices")
print(f"Test set: {X_test_dev.shape[0]:,} devices")
print(f"\nFeatures:")
print("  1. Vdd_voltage     - Supply voltage (V)")
print("  2. Idd_current_mA  - Supply current (mA)")
print("  3. frequency_MHz   - Operating frequency (MHz)")
print("  4. power_mW        - Power consumption (mW)")
print("  5. temperature_C   - Junction temperature (°C)")
print(f"\nClass Distribution:")
print(f"  PASS: {np.sum(y_train_dev == 1):,} devices ({np.mean(y_train_dev == 1)*100:.1f}%)")
print(f"  FAIL: {np.sum(y_train_dev == 0):,} devices ({np.mean(y_train_dev == 0)*100:.1f}%)")

# Train Naive Bayes classifier
import time

start_time = time.time()
nb_device = GaussianNB()
nb_device.fit(X_train_dev, y_train_dev)
train_time = time.time() - start_time

# Predictions with timing
start_time = time.time()
y_pred_dev = nb_device.predict(X_test_dev)
y_proba_dev = nb_device.predict_proba(X_test_dev)
pred_time = time.time() - start_time

# Evaluate
accuracy_dev = accuracy_score(y_test_dev, y_pred_dev)
precision_dev = precision_score(y_test_dev, y_pred_dev)
recall_dev = recall_score(y_test_dev, y_pred_dev)
f1_dev = f1_score(y_test_dev, y_pred_dev)
conf_matrix = confusion_matrix(y_test_dev, y_pred_dev)

print("\n" + "=" * 70)
print("PERFORMANCE RESULTS")
print("=" * 70)
print(f"\nTraining Time: {train_time*1000:.2f} ms")
print(f"Prediction Time: {pred_time*1000:.2f} ms ({X_test_dev.shape[0]/pred_time:.0f} devices/sec)")
print(f"\n{'Metric':<25} {'Score':<10} {'Business Impact':<30}")
print("-" * 70)
print(f"{'Accuracy':<25} {accuracy_dev:.4f}     {'Overall screening reliability':<30}")
print(f"{'Precision':<25} {precision_dev:.4f}     {'PASS prediction confidence':<30}")
print(f"{'Recall':<25} {recall_dev:.4f}     {'True PASS capture rate':<30}")
print(f"{'F1-Score':<25} {f1_dev:.4f}     {'Balanced performance':<30}")

print("\n" + "=" * 70)
print("CONFUSION MATRIX")
print("=" * 70)
print(f"\nActual vs Predicted:")
print(f"                 Predicted FAIL  Predicted PASS")
print(f"Actual FAIL      {conf_matrix[0,0]:<15} {conf_matrix[0,1]:<15}")
print(f"Actual PASS      {conf_matrix[1,0]:<15} {conf_matrix[1,1]:<15}")

# Calculate business metrics
false_pass = conf_matrix[0, 1]  # Predicted PASS but actually FAIL (worst case)
false_fail = conf_matrix[1, 0]  # Predicted FAIL but actually PASS (yield loss)

print(f"\n📊 Business Impact:")
print(f"  • False PASS (defects shipped): {false_pass} devices ({false_pass/len(y_test_dev)*100:.2f}%)")
print(f"  • False FAIL (yield loss): {false_fail} devices ({false_fail/len(y_test_dev)*100:.2f}%)")
print(f"  • Cost of False PASS: ${false_pass * 50:,} (assume $50/RMA)")
print(f"  • Cost of False FAIL: ${false_fail * 10:,} (assume $10/device)")

# Probability-based confidence scoring
high_conf_threshold = 0.9
low_conf_threshold = 0.6

high_conf_pass = np.sum(y_proba_dev[:, 1] > high_conf_threshold)
low_conf = np.sum((y_proba_dev[:, 1] > low_conf_threshold) & (y_proba_dev[:, 1] < high_conf_threshold))
borderline = np.sum((y_proba_dev[:, 1] <= low_conf_threshold) & (y_proba_dev[:, 1] >= 1-low_conf_threshold))

print(f"\n🎯 Confidence-Based Triage:")
print(f"  • High confidence PASS (P>0.9): {high_conf_pass} devices → Ship immediately")
print(f"  • Medium confidence: {low_conf} devices → Standard flow")
print(f"  • Borderline (0.4<P<0.6): {borderline} devices → SVM refinement needed")

print("\n" + "=" * 70)

## 🚀 Real-World Project Ideas

### Post-Silicon Validation Projects

#### 1. **Real-Time Test Floor Screening System**
**Objective**: Build production-ready pass/fail classifier for high-volume device testing

**Features**:
- Stream test data from ATE (Automated Test Equipment) via STDF format
- Real-time Naive Bayes classification (<10ms latency requirement)
- Confidence-based routing: High confidence → ship, Low confidence → retest, Borderline → SVM
- Adaptive learning: Retrain on recent failures weekly
- Dashboard: Throughput (devices/hour), accuracy trends, false positive rate

**Success Metrics**:
- 95%+ accuracy with <10ms inference time
- Reduce test time by 30% (skip deep tests for high-confidence PASS)
- False pass rate <0.5% (critical for quality)

**Business Value**: $2M+ annual savings from faster throughput + fewer escapes

---

#### 2. **Multi-Class Binning Classifier**
**Objective**: Classify devices into 5+ performance bins (Premium, Standard, Value, Fail-Electrical, Fail-Thermal)

**Features**:
- Train separate Naive Bayes for each bin pair (one-vs-rest strategy)
- Feature engineering: Derive ratios (Idd/Frequency), temperature coefficients
- Calibrate probabilities using Platt scaling for better confidence estimates
- Handle class imbalance (premium bins rare) with SMOTE or class weights

**Success Metrics**:
- 90%+ accuracy across all bins
- Maximize premium bin yield ($$$ value)
- Minimize bin-crossing errors (Premium→Fail is worst)

**Business Value**: $10M+ revenue from optimized binning strategy

---

#### 3. **Wafer-Level Spatial Pattern Detection**
**Objective**: Use Naive Bayes with spatial features to identify wafer map failure patterns

**Features**:
- Features: Die (x,y) position, nearest neighbor status, radial distance from center
- Multinomial NB for pattern types: Edge fail, Center fail, Scratch, Random
- Ensemble with K-Means for unsupervised pattern discovery
- Visualize probability heatmaps overlaid on wafer maps

**Success Metrics**:
- 85%+ pattern type accuracy
- Early detection (within 10% of wafer completion)
- Root cause correlation (process step identification)

**Business Value**: $5M+ savings from early process intervention

---

#### 4. **Failure Mode Classification from Parametric Trends**
**Objective**: Predict failure mode (Open, Short, Leakage, Timing) from parametric test data

**Features**:
- Multivariate Gaussian NB with correlated features (e.g., Vdd + Idd correlation)
- Temporal features: Parameter drift over test sequence
- Compare Gaussian vs Multinomial NB for different failure types
- Integrate with failure analysis (FA) database for ground truth

**Success Metrics**:
- 80%+ failure mode classification accuracy
- Reduce FA turnaround time by 50% (pre-diagnosis)
- Enable automated root cause analysis workflows

**Business Value**: $3M+ savings from faster debug cycles

---

### General AI/ML Projects

#### 5. **Real-Time Email Spam Classifier**
**Objective**: Build production spam filter using Multinomial Naive Bayes

**Features**:
- TF-IDF vectorization + Multinomial NB for text classification
- Incremental learning: Update model with user feedback (spam/not spam)
- Personalization: Per-user models learn individual preferences
- Handle HTML emails, attachments, sender reputation features

**Success Metrics**:
- 98%+ spam detection rate
- <0.1% false positive rate (legit emails marked spam)
- <5ms classification latency

**Business Value**: Protect 100K+ users from phishing, productivity gains

---

#### 6. **Medical Diagnosis Support System**
**Objective**: Assist doctors with preliminary diagnosis from symptoms using Naive Bayes

**Features**:
- Features: Patient symptoms (binary presence/absence), age, vitals
- Bernoulli NB for symptom presence, Gaussian NB for continuous vitals
- Output: Top 3 probable diagnoses with confidence scores
- Integrate with medical knowledge base for differential diagnosis

**Success Metrics**:
- 85%+ top-3 accuracy (correct diagnosis in top 3 suggestions)
- Reduce misdiagnosis risk by providing probability rankings
- HIPAA compliance for patient data

**Business Value**: Improve diagnostic accuracy, reduce healthcare costs

---

#### 7. **Sentiment Analysis for Customer Reviews**
**Objective**: Classify product reviews as Positive/Negative/Neutral using Naive Bayes

**Features**:
- Text preprocessing: Tokenization, stopword removal, lemmatization
- Multinomial NB on word counts / TF-IDF features
- Handle negation ("not good" → negative context)
- Aspect-based sentiment: Extract sentiment per product aspect (battery, screen, camera)

**Success Metrics**:
- 90%+ sentiment classification accuracy
- Real-time processing for 1M+ reviews/day
- Actionable insights: Identify product improvement areas

**Business Value**: $1M+ revenue from product quality insights

---

#### 8. **Fraud Detection for Financial Transactions**
**Objective**: Real-time fraud classification for credit card transactions

**Features**:
- Features: Transaction amount, merchant category, time of day, location, user history
- Gaussian NB for continuous features (amount), Categorical NB for discrete features
- Hybrid ensemble: Naive Bayes (fast screening) + SVM (borderline cases)
- Handle class imbalance (fraud <<1%) with cost-sensitive learning

**Success Metrics**:
- 95%+ fraud detection rate
- <1% false positive rate (legitimate transactions blocked)
- <10ms decision latency for real-time authorization

**Business Value**: $50M+ annual fraud prevention, customer trust

---

## 📊 Naive Bayes Variant Comparison

| Variant | Data Type | Use Case | Assumption | Example |
|---------|-----------|----------|------------|---------|
| **Gaussian NB** | Continuous features | Device testing, medical vitals | Features ~ Normal distribution | Vdd, Idd, frequency |
| **Multinomial NB** | Count features | Text classification, word counts | Discrete counts | Email spam, sentiment |
| **Bernoulli NB** | Binary features | Symptom presence, feature flags | Binary 0/1 values | Has_fever, Has_cough |
| **Categorical NB** | Categorical features | Survey responses, categories | Discrete categories | Color, Size, Type |
| **Complement NB** | Imbalanced text | Rare class text problems | Corrects for imbalance | Rare disease diagnosis |

## 🎯 Key Takeaways

### When to Use Naive Bayes

✅ **BEST FOR:**
- **Text classification** (spam, sentiment, document categorization)
- **Real-time predictions** (low latency critical)
- **Small training datasets** (works with limited data)
- **Baseline models** (fast to implement and iterate)
- **Probability estimates needed** (risk-based decisions)
- **High-dimensional data** (many features, scales well)
- **Streaming/online learning** (easy to update incrementally)
- **Post-silicon**: Fast parametric screening, initial triage

❌ **AVOID WHEN:**
- **Features are highly correlated** (violates independence assumption)
- **Decision boundaries are complex/non-linear** (use SVM, neural nets)
- **Maximum accuracy required** (Naive Bayes trades accuracy for speed)
- **Feature interactions critical** (e.g., x1*x2 matters, not just x1 and x2)
- **Small number of samples per class** (insufficient for reliable statistics)

---

### Algorithm Comparison

| Aspect | Naive Bayes | Logistic Regression | SVM | Decision Trees |
|--------|-------------|---------------------|-----|----------------|
| **Training Speed** | ⭐⭐⭐⭐⭐ Instant | ⭐⭐⭐⭐ Fast | ⭐⭐ Slow | ⭐⭐⭐ Medium |
| **Prediction Speed** | ⭐⭐⭐⭐⭐ Fastest | ⭐⭐⭐⭐ Fast | ⭐⭐⭐ Medium | ⭐⭐⭐⭐ Fast |
| **Accuracy** | ⭐⭐⭐ Good | ⭐⭐⭐⭐ Very Good | ⭐⭐⭐⭐⭐ Best | ⭐⭐⭐⭐ Very Good |
| **Interpretability** | ⭐⭐⭐⭐⭐ Excellent | ⭐⭐⭐⭐ Good | ⭐⭐⭐ Medium | ⭐⭐⭐⭐⭐ Excellent |
| **Handles Correlated Features** | ❌ No | ✅ Yes | ✅ Yes | ✅ Yes |
| **Probability Estimates** | ✅ Well-calibrated | ✅ Well-calibrated | ⚠️ Needs calibration | ⚠️ Needs calibration |
| **Memory Usage** | ⭐⭐⭐⭐⭐ Tiny | ⭐⭐⭐⭐ Small | ⭐⭐ Large | ⭐⭐⭐ Medium |
| **Online Learning** | ✅ Easy | ✅ Possible | ❌ Difficult | ⭐⭐⭐ Possible |

---

### Strengths

1. **Blazing Fast Training**: Just compute means/variances per class (closed-form solution)
2. **Real-Time Inference**: Evaluate Gaussian PDF + sum logs → sub-millisecond predictions
3. **Probability Calibration**: Unlike SVM, outputs are actual probabilities (good for risk-based decisions)
4. **Scales to High Dimensions**: Works well with thousands of features (text, genomics)
5. **Small Data Friendly**: Can work with limited training samples
6. **Simple to Implement**: Minimal hyperparameters, easy to understand and debug
7. **Incremental Learning**: Easy to update with new data without full retraining
8. **Robust to Irrelevant Features**: Irrelevant features get low likelihoods, minimal impact

---

### Limitations

1. **"Naive" Assumption Rarely Holds**: Real-world features are often correlated
   - **Example**: In text, "machine" and "learning" co-occur (not independent)
   - **Impact**: Can underperform when correlations are strong
   - **Mitigation**: Feature selection to remove redundant/correlated features

2. **Gaussian Assumption May Be Wrong**: Real data may not be normally distributed
   - **Example**: Device test data may have multimodal or skewed distributions
   - **Impact**: Poor class boundaries if distributions are non-Gaussian
   - **Mitigation**: Transform features (log, Box-Cox), use Kernel Density Estimation

3. **Zero Frequency Problem**: If a feature never appears with a class in training
   - **Example**: Word "viagra" never appears in ham emails during training
   - **Impact**: P(word|ham) = 0 → entire posterior becomes 0
   - **Mitigation**: Laplace smoothing (add pseudo-counts)

4. **Sensitive to Irrelevant Features**: While robust, too many noise features hurt
   - **Example**: Including random features dilutes signal
   - **Impact**: Accumulation of small errors across many features
   - **Mitigation**: Feature selection, regularization

5. **Cannot Learn Feature Interactions**: Treats all features independently
   - **Example**: Can't learn that high_Vdd AND high_Idd together indicate failure
   - **Impact**: Misses combinatorial patterns
   - **Mitigation**: Manually engineer interaction features (x1*x2, x1/x2)

6. **Class Imbalance Issues**: Priors dominate when classes are heavily imbalanced
   - **Example**: 99% PASS, 1% FAIL → model predicts PASS for everything
   - **Impact**: Poor minority class recall
   - **Mitigation**: Cost-sensitive learning, SMOTE, Complement Naive Bayes

---

### Best Practices

**1. Feature Engineering**
- Remove highly correlated features (correlation > 0.9)
- Transform non-Gaussian features (log, sqrt, Box-Cox)
- Create interaction features for known dependencies
- Normalize/standardize continuous features for better Gaussian fit

**2. Variant Selection**
- **Gaussian NB**: Continuous features (test parameters, sensor data)
- **Multinomial NB**: Count data (word frequencies, event counts)
- **Bernoulli NB**: Binary features (symptom presence, flags)
- **Complement NB**: Imbalanced text classification

**3. Hyperparameter Tuning**
- `var_smoothing`: Add to variances for stability (default 1e-9, try 1e-8 to 1e-10)
- Laplace `alpha`: For Multinomial/Bernoulli (default 1.0, try 0.1 to 10)
- `class_prior`: Can specify manually if training distribution ≠ production distribution

**4. Model Validation**
- **Check Gaussian Assumption**: Plot feature distributions per class, Q-Q plots
- **Inspect Learned Parameters**: Verify means/variances make sense domain-wise
- **Probability Calibration**: Use calibration curves to validate probability estimates
- **Cross-Validation**: Essential for small datasets, use stratified k-fold

**5. Production Deployment**
- **Hybrid Approach**: Naive Bayes for fast screening + SVM for borderline cases
- **Confidence Thresholding**: Route low-confidence predictions to human review
- **Incremental Updates**: Retrain periodically with new data (weekly/monthly)
- **Monitor Drift**: Track accuracy over time, retrain if performance degrades

---

### Probability Calibration Check

Naive Bayes probabilities are generally well-calibrated, but verify:
- **Calibration Curve**: Plot predicted probability vs actual frequency
- **Expected**: Should lie on diagonal (predicted 70% → 70% actual positive rate)
- **If Miscalibrated**: Use Platt scaling or isotonic regression post-training

---

### Computational Complexity

- **Training**: $O(n \cdot d \cdot k)$ where n=samples, d=features, k=classes
  - Just compute means/variances → very fast
- **Prediction**: $O(d \cdot k)$ per sample
  - Evaluate Gaussian PDF for each feature-class pair
- **Memory**: $O(d \cdot k)$ to store means/variances
  - Tiny compared to SVM's support vectors

**Example**: 1M samples, 1000 features, 10 classes
- Training: <1 second
- Prediction: <0.01ms per sample
- Memory: ~80KB (1000 features × 10 classes × 8 bytes)

---

### Post-Silicon Validation Best Practices

1. **Fast Screening + Precise Refinement**: Naive Bayes (1st pass) → SVM (borderline cases)
2. **Probability-Based Routing**: 
   - P(PASS) > 0.95 → Ship immediately
   - 0.5 < P(PASS) < 0.95 → Standard retest
   - P(PASS) < 0.5 → Deep analysis
3. **Feature Selection**: Use parametric tests most correlated with failures
4. **Adaptive Learning**: Retrain weekly on latest failures to adapt to process drift
5. **Ensemble with Domain Rules**: Combine NB probabilities with hard limits (e.g., Vdd < 1.7V → FAIL)

---

### Next Steps in Learning Path

**Completed**: Naive Bayes (probabilistic classification)

**Next**: 
- **026 K-Means Clustering** - Unsupervised learning for wafer map pattern discovery
- **027 Hierarchical Clustering** - Dendrograms for device similarity analysis
- **028 DBSCAN** - Density-based failure hotspot detection

**Advanced Topics**:
- **Gaussian Mixture Models (GMM)** - Soft clustering with probabilistic assignments
- **Hidden Markov Models (HMM)** - Sequential data modeling (test sequence analysis)
- **Bayesian Networks** - Model feature dependencies (relax naive assumption)

---

### References & Further Reading

**Theory**:
- Pattern Recognition and Machine Learning (Bishop) - Chapter 4
- The Elements of Statistical Learning (Hastie et al.) - Section 6.6.3

**sklearn Documentation**:
- [GaussianNB](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html)
- [MultinomialNB](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html)
- [BernoulliNB](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.BernoulliNB.html)

**Papers**:
- "Naive Bayes at Forty" (Lewis, 1998) - Historical perspective
- "On Discriminative vs. Generative Classifiers" (Ng & Jordan, 2002)

---

## 🎉 Congratulations!

You now understand:
✅ Bayes' theorem and probabilistic classification  
✅ The "naive" independence assumption and its implications  
✅ Gaussian, Multinomial, and Bernoulli variants  
✅ From-scratch implementation of Naive Bayes  
✅ Production deployment with sklearn  
✅ When to use Naive Bayes vs. SVM/other classifiers  
✅ Real-world applications in post-silicon validation  

**Key Insight**: Naive Bayes trades accuracy for speed and simplicity. Use it for fast screening, baseline models, and when probability estimates matter. For maximum accuracy with complex boundaries, use SVM or neural networks. In production, combine both: Naive Bayes for 95% of cases (fast), SVM for the 5% borderline cases (precise). 🚀